In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from core.load_data import load_data, initalize_result_df

from core.preprocessing import decoding_numbers, train_val_test_split
from core.preprocessing import list_resampler, all_columns

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from imblearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
#ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
path = '../data/raw/2011 Census Microdata Teaching File.csv'
df = pd.read_csv(path)
df_decoded = decoding_numbers(df)


/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_44884/512910874.py:2: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


### Train Test Validation Split


In [3]:
features_train, target_train, features_val, target_val, features_test, target_test = train_val_test_split(df_decoded.iloc[:,:-1], df_decoded.iloc[:,-1])

Resample the data to speed up the calculations.

In [4]:
features_train_resampled = features_train.sample(n=10000, axis = 0, random_state=42)
target_train_resampled = target_train.sample(n=10000, axis = 0, random_state=42)

Initialize dataframe for results.

In [6]:
initalize_result_df()
path_results = '../data/processed/Comparison_models.csv'
results = pd.read_csv(path_results)#, index_col = 'index')


In [ ]:
#results.to_csv('../data/processed/results2_till_logreg.csv')

## Functions

In [7]:
def build_pipeline(model, number_resampler = 0):
    """Build a pipeline with the model, a one-hot encoder and a resampler."""
    pipeline = Pipeline([('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore')), 
                               list_resampler[number_resampler],
                               ('model', model)])
    return pipeline

In [8]:
def fit_and_predict(pipeline, columns):
   """Fit the given pipeline with the given columns and return the pipeline"""
   pipeline.fit(features_train_resampled.loc[:, columns], target_train_resampled)
   target_train_pred = pipeline.predict(features_train_resampled.loc[:,columns])
   target_val_pred = pipeline.predict(features_val.loc[:,columns])
   acc_train = accuracy_score(target_train_resampled, target_train_pred)
   acc_val = accuracy_score(target_val, target_val_pred)
   #print('Accuracy of train set: ', acc_train)
   #print('Accuracy of validation set: ', acc_val)
   data = {
   "index": [-1],
    "columns": ['Features', 'Number_features', 'Model_typ', 'Accuracy_train', 'Accuracy_val', 'Model', 'search_space', 'search_res'],
    "data": [[columns, len(columns), pipeline.named_steps['model'], acc_train, acc_val, pipeline, {}, None]],
    "index_names": ["index"],
    "column_names": ["Columns"],
    }
   return pipeline, data

## Models


In [9]:
models = [DecisionTreeClassifier(),
          RandomForestClassifier(random_state=42, n_estimators=5), 
          LogisticRegression(solver='lbfgs'),
          KNeighborsClassifier(),
          LinearSVC(multi_class='crammer_singer')
          ]

In [10]:
for model in models:
    #print(model)
    for col in all_columns:
        pipeline = build_pipeline(model, number_resampler = 0)
        pipeline_fitted, data = fit_and_predict(pipeline, [col])
        results = pd.concat([results, pd.DataFrame.from_dict(data, orient="tight")])
        results = results.reset_index(drop=True)
    results.to_csv('../data/processed/Comparison_models.csv', index=False)

/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_44884/254255647.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame.from_dict(data, orient="tight")])
/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, i

### Artificial Neural Network


In [11]:
dict_social_grade = {'AB': 4, 'C2': 3, 'C1': 2, 'DE': 1, 'No code required': 0}

In [12]:
target_train = target_train.replace(dict_social_grade)
target_train_resampled = target_train_resampled.replace(dict_social_grade)
target_val = target_val.replace(dict_social_grade)

/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_44884/882130119.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target_train = target_train.replace(dict_social_grade)
/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_44884/882130119.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target_train_resampled = target_train_resampled.replace(dict_social_grade)
/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_44884/882130119.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will

In [13]:
num_classes = 5
target_train_resampled_encoded = tf.keras.utils.to_categorical(target_train_resampled, num_classes=num_classes)
target_val_encoded  = tf.keras.utils.to_categorical(target_val, num_classes=num_classes)

pipeline_an = Pipeline(steps = [('ohe', OneHotEncoder(sparse_output=False, 
                                                      handle_unknown='ignore'))])
for col in all_columns:
    col = [col]
    feat_train_transf = pipeline_an.fit_transform(features_train_resampled.loc[:,col])
    feat_val_transf = pipeline_an.transform(features_val.loc[:,col])
    #feat_test_transf = pipeline_an.transform(features_test)

    #resampler = SMOTE(random_state=42) 
    #resampler = RandomUnderSampler(random_state=42)
    #resampler = RandomOverSampler(random_state=42)
    #feat_train_res, tar_train_res = resampler.fit_resample(feat_train_transf, target_train)
    feat_train_res = feat_train_transf
    tar_train_res = target_train
    model_an = Sequential()
    model_an.add(Dense(1, activation='sigmoid', input_dim=feat_train_res.shape[1]))
    model_an.add(Dense(5, activation = 'relu'))
    model_an.add(Dropout(rate=0.3))
    model_an.add(Dense(num_classes, activation='softmax'))

    model_an.compile(optimizer = "adam", loss = "categorical_crossentropy", 
                     metrics=['accuracy'])  # compile the model

    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 2)
    hist = model_an.fit(feat_train_res, 
                    target_train_resampled_encoded, 
                    validation_data=(feat_val_transf, target_val_encoded),
                    epochs=200, 
                    batch_size=32,
                    callbacks=[early_stop]
                   )
    test_loss, acc_val = model_an.evaluate(pipeline_an.transform(features_val.loc[:,col]),
                                       target_val_encoded, verbose=2)
    test_loss, acc_train = model_an.evaluate(pipeline_an.transform(features_train_resampled.loc[:,col]),
                                       target_train_resampled_encoded, verbose=2)
    data = {
   "index": [-1],
    "columns": ['Features', 'Number_features', 'Model_typ', 'Accuracy_train', 'Accuracy_val', 'Model', 'search_space', 'search_res'],
    "data": [[col, len(col), 'ANN', acc_train, acc_val, model_an, {}, None]],
    "index_names": ["index"],
    "column_names": ["Columns"],
    }
    results = pd.concat([results, pd.DataFrame.from_dict(data, orient="tight")])
    results = results.reset_index(drop=True)
results.to_csv('../data/processed/Comparison_models.csv', index=False)
    

Epoch 1/200


/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2552 - loss: 1.6008 - val_accuracy: 0.2789 - val_loss: 1.5795
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2729 - loss: 1.5821 - val_accuracy: 0.2789 - val_loss: 1.5780
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2688 - loss: 1.5792 - val_accuracy: 0.2789 - val_loss: 1.5768
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2746 - loss: 1.5783 - val_accuracy: 0.2789 - val_loss: 1.5767
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2812 - loss: 1.5770 - val_accuracy: 0.2789 - val_loss: 1.5766
3205/3205 - 1s - 263us/step - accuracy: 0.2789 - loss: 1.5766
313/313 - 0s - 309us/step - accuracy: 0.2829 - loss: 1.5749
Epoch 1/200


/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2584 - loss: 1.5796 - val_accuracy: 0.2978 - val_loss: 1.5716
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2915 - loss: 1.5726 - val_accuracy: 0.2789 - val_loss: 1.5702
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2917 - loss: 1.5699 - val_accuracy: 0.2978 - val_loss: 1.5685
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2927 - loss: 1.5692 - val_accuracy: 0.2978 - val_loss: 1.5666
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2977 - loss: 1.5673 - val_accuracy: 0.2978 - val_loss: 1.5653
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2989 - loss: 1.5642 - val_accuracy: 0.2978 - val_loss: 1.5614
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2985 - loss: 1.5616 - val_accuracy: 0.2978 - val_loss: 1.5584
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2989 - loss: 1.5584 - val_accuracy: 0.2978

...

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2613 - loss: 1.5655 - val_accuracy: 0.2789 - val_loss: 1.5391
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2886 - loss: 1.5282 - val_accuracy: 0.2789 - val_loss: 1.4917
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3351 - loss: 1.4789 - val_accuracy: 0.4544 - val_loss: 1.4302
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3399 - loss: 1.4167 - val_accuracy: 0.4544 - val_loss: 1.3591
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3781 - loss: 1.3722 - val_accuracy: 0.4544 - val_loss: 1.3090
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4174 - loss: 1.3388 - val_accuracy: 0.4544 - val_loss: 1.2745
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4345 - loss: 1.3106 - val_accuracy: 0.4544 - val_loss: 1.2501
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4344 - loss: 1.3013 - val_accuracy: 0.4544

/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2564 - loss: 1.5872 - val_accuracy: 0.2789 - val_loss: 1.5746
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2824 - loss: 1.5722 - val_accuracy: 0.2789 - val_loss: 1.5661
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2974 - loss: 1.5590 - val_accuracy: 0.2789 - val_loss: 1.5405
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3477 - loss: 1.5136 - val_accuracy: 0.4397 - val_loss: 1.4662
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4061 - loss: 1.4365 - val_accuracy: 0.4544 - val_loss: 1.3754
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4304 - loss: 1.3829 - val_accuracy: 0.4544 - val_loss: 1.3246
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4345 - loss: 1.3608 - val_accuracy: 0.4544 - val_loss: 1.3004
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4358 - loss: 1.3473 - val_accuracy: 0.4544

/Users/qp251849/portfolio_template/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2697 - loss: 1.5834 - val_accuracy: 0.2789 - val_loss: 1.5650
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3158 - loss: 1.5575 - val_accuracy: 0.2789 - val_loss: 1.5441
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3262 - loss: 1.5343 - val_accuracy: 0.3805 - val_loss: 1.5165
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3270 - loss: 1.5183 - val_accuracy: 0.3805 - val_loss: 1.4980
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3254 - loss: 1.5055 - val_accuracy: 0.3805 - val_loss: 1.4855
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3280 - loss: 1.4981 - val_accuracy: 0.3805 - val_loss: 1.4756
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3290 - loss: 1.4941 - val_accuracy: 0.3805 - val_loss: 1.4693
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3248 - loss: 1.4929 - val_accuracy: 0.3805